# 변경된 확률표를 사용하였습니다.

In [1]:
import numpy as np
import pandas as pd

mat = [[0 for i in range(24)] for i in range(24)] 


    #전, 현, 후, 파괴확률
k = [[0, 0.05, 0.95, 0],#0
     [0, 0.1,0.9,0],#1
     [0, 0.15,0.85,0],#2
     [0, 0.15,0.85,0],#3
     [0, 0.2,0.8,0],#4
     [0,0.25,0.75,0],#5
     [0,0.3,0.7,0],#6
     [0,0.35,0.65,0],#7
     [0,0.4,0.6,0],#8
     [0,0.45,0.55,0], #9
     [0,0.5,0.5,0], #10
     [0.55,0,0.45,0], #11
     [0.594,0,0.4,0.006], #12
     [0.637,0,0.35,0.013], #13
     [0.686,0,0.3,0.014], #14
     [0,0.679,0.3,0.021], #15
     [0.679, 0,0.3,0.021], #16
     [0.679, 0,0.3,0.021], #17
     [0.672, 0,0.3,0.028], #18
     [0.672, 0,0.3,0.028], #19
     [0,0.63,0.3,0.07], #20
     [0.63,0,0.3,0.07]] #21
     

for i in range(22):
    lst = k[i]
    for j in range(len(lst)):
        if lst[j] == 0:
            continue
        else:
            if j == 0:
                mat[i][i-1] = lst[j]
            elif j == 1:
                mat[i][i] = lst[j]
            elif j == 2:
                mat[i][i+1] = lst[j]
            else:
                mat[i][-1] = lst[j]

P = np.array(mat)

mat = pd.DataFrame(P)
T = np.array(mat.iloc[:22,:22])
R = np.array(mat.iloc[:22:,-2:])
R

array([[0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.006],
       [0.   , 0.013],
       [0.   , 0.014],
       [0.   , 0.021],
       [0.   , 0.021],
       [0.   , 0.021],
       [0.   , 0.028],
       [0.   , 0.028],
       [0.   , 0.07 ],
       [0.3  , 0.07 ]])

# 스타캐치만을 고려한 모델(전이확률행렬)

In [2]:
## 스타캐치를 하게 될 경우 파괴확률은 그대로, 상승확률만 곱연산으로 올라간다고 가정.
## 11성->12성부터 전부 스타캐치 계속 성공한다고 가정한 상황

mat = [[0 for i in range(24)] for i in range(24)] 

    #전, 현, 후, 파괴확률
k = [[0, 0.05, 0.95, 0],#0
     [0, 0.1,0.9,0],#1
     [0, 0.15,0.85,0],#2
     [0, 0.15,0.85,0],#3
     [0, 0.2,0.8,0],#4
     [0,0.25,0.75,0],#5
     [0,0.3,0.7,0],#6
     [0,0.35,0.65,0],#7
     [0,0.4,0.6,0],#8
     [0,0.45,0.55,0], #9
     [0,0.5,0.5,0], #10
     [1-0.45*1.045,0,0.45*1.045,0], #11
     [1-0.4*1.045-0.006,0,0.4*1.045,0.006], #12
     [1-0.35*1.045-0.013,0,0.35*1.045,0.013], #13
     [1-0.3*1.045-0.014,0,0.3*1.045,0.014], #14
     [0,1-0.3*1.045-0.021,0.3*1.045,0.021], #15
     [1-0.3*1.045-0.021, 0,0.3*1.045,0.021], #16
     [1-0.3*1.045-0.021, 0,0.3*1.045,0.021], #17
     [1-0.3*1.045-0.028, 0,0.3*1.045,0.028], #18
     [1-0.3*1.045-0.028, 0,0.3*1.045,0.028], #19
     [0,1-0.3*1.045-0.07,0.3*1.045,0.07], #20
     [1-0.3*1.045-0.07,0,0.3*1.045,0.07]] #21

for i in range(22):
    lst = k[i]
    for j in range(len(lst)):
        if lst[j] == 0:
            continue
        else:
            if j == 0:
                mat[i][i-1] = lst[j]
            elif j == 1:
                mat[i][i] = lst[j]
            elif j == 2:
                mat[i][i+1] = lst[j]
            else:
                mat[i][-1] = lst[j]

P = np.array(mat)

mat = pd.DataFrame(P)
T = np.array(mat.iloc[:22,:22])
R = np.array(mat.iloc[:22:,-2:])

I = np.eye(len(T))
E = np.linalg.inv(I-T)
e = np.ones(len(T)).reshape(-1,1)

# 시뮬레이션을 통한 estimation

> 1,000회 실시

In [3]:
## 찬스타임 고려한 상황 하

def check(a,state):
    p = 0
    for i in range(4):
        p += k[state][i]
        if a>p:
            continue
        else:
            return i

In [14]:
ans = []
state_num = [0 for i in range(22)]
state_num[0] = 1

for i in range(1000):
    state = 0
    trial = 0
    flag = 0
    chance_time = 0
    while (state != -1) and (state != 22):
        trial += 1
        prob = np.random.rand(1)
        b = check(prob,state)
    if flag == 2:
        state += 1
        flag = 0
        chance_time += 1

    elif b == 0:
        state -= 1
        flag += 1

    elif b == 1:
        flag = 0
        pass

    elif b == 2:
        flag = 0
        state += 1

    else:
        state = -1

    if state != -1 and state != 22:
        state_num[state] += 1
    #print(state)
    #print(flag)

    ans.append([state, chance_time, trial])

KeyboardInterrupt: 

In [9]:
# 1,000번 test했을 때 성공 횟수
cnt = 0
for i in ans:
    if i[0] == 22:
        cnt+=1
    else:
        continue

print(cnt)

100


In [10]:
# 1,000번 test했을 때 각 state 평균 방문 횟수
np.array(state_num)/1000

array([ 0.048,  1.12 ,  1.169,  1.19 ,  1.231,  1.325,  1.467,  1.565,
        1.684,  1.839, 11.187, 12.452, 11.318,  7.072,  2.555, 15.225,
        8.648,  4.588,  2.039,  0.633,  0.996,  0.311])

In [11]:
# 평균 찬스타임 횟수
print(sum(np.array(ans)[:1000,1])/1000)

# 평균 trial 횟수
print(sum(np.array(ans)[:1000,2])/1000)

7.497
90.661


> 10,000회 실시

In [13]:
ans = []
state_num = [0 for i in range(22)]
state_num[0] = 1

for i in range(10000):
    state = 0
    trial = 0
    flag = 0
    chance_time = 0
    while (state != -1) and (state != 22):
        trial += 1
        prob = np.random.rand(1)
        b = check(prob,state)
    if flag == 2:
        state += 1
        flag = 0
        chance_time += 1

    elif b == 0:
        state -= 1
        flag += 1

    elif b == 1:
        flag = 0
        pass

    elif b == 2:
        flag = 0
        state += 1

    else:
        state = -1

    if state != -1 and state != 22:
        state_num[state] += 1
    #print(state)
    #print(flag)

    ans.append([state, chance_time, trial])

KeyboardInterrupt: 

In [ ]:
# 10,000번 test했을 때 성공 횟수
cnt = 0
for i in ans:
    if i[0] == 22:
        cnt+=1
    else:
        continue

print(cnt)

In [ ]:
# 10,000번 test했을 때 각 state 평균 방문 횟수
np.array(state_num)/10000

In [ ]:
# 평균 찬스타임 횟수
print(sum(np.array(ans)[:,1])/10000)

# 평균 trial 횟수
print(sum(np.array(ans)[:,2])/10000)

> 100,000회 실시

In [ ]:
ans = []
state_num = [0 for i in range(22)]
state_num[0] = 1

for i in range(100000):
    state = 0
    trial = 0
    flag = 0
    chance_time = 0
    while (state != -1) and (state != 22):
        trial += 1
        prob = np.random.rand(1)
        b = check(prob,state)
    if flag == 2:
        state += 1
        flag = 0
        chance_time += 1

    elif b == 0:
        state -= 1
        flag += 1

    elif b == 1:
        flag = 0
        pass

    elif b == 2:
        flag = 0
        state += 1

    else:
        state = -1

    if state != -1 and state != 22:
        state_num[state] += 1
    #print(state)
    #print(flag)

    ans.append([state, chance_time, trial])

In [ ]:
# 100,000번 test했을 때 성공 횟수
cnt = 0
for i in ans:
    if i[0] == 22:
        cnt+=1
    else:
        continue

print(cnt)

In [ ]:
# 100,000번 test했을 때 각 state 평균 방문 횟수
np.array(state_num)/100000

In [ ]:
# 10만번 평균 찬스타임 횟수
print(sum(np.array(ans)[:,1])/100000)

# 10만번 평균 trial 횟수
print(sum(np.array(ans)[:,2])/100000)

> 흡수되기까지 평균적으로 사용한 금액

In [ ]:
cost = np.array([164800,328700,492500,656400,820200,984000,1147900,1311700,1475600,1639400,6639400,
        8397300,10422900,12731500,15338200,36514500,43008300,50185100,58072700,66698700,
        76090000,86273300]).reshape(-1,1)

# 1,000회
arr1 = np.array([ 0.068,  1.125,  1.156,  1.163,  1.258,  1.331,  1.427,  1.521,
        1.689,  1.83 , 11.235, 12.72 , 11.59 ,  7.296,  2.591, 14.095,
        7.931,  4.212,  1.942,  0.606,  1.107,  0.321])

# 10,000회
arr2 = np.array([ 0.0537,  1.114 ,  1.1782,  1.1747,  1.2562,  1.3318,  1.4366,
        1.5472,  1.6671,  1.8219, 11.0838, 12.4291, 11.2999,  7.1565,
        2.6016, 15.1167,  8.4353,  4.3995,  1.9517,  0.6103,  1.0432,
        0.3193])

# 100,000회
arr3 = np.array([ 0.05251,  1.11066,  1.17845,  1.1757 ,  1.2529 ,  1.33095,
        1.43053,  1.54428,  1.66975,  1.8242 , 11.07139, 12.43727,
       11.27138,  7.10349,  2.59616, 15.12696,  8.4802 ,  4.43817,
        1.97898,  0.62024,  1.02477,  0.3233 ])

print(np.matmul(arr1,cost))
print(np.matmul(arr2,cost))
print(np.matmul(arr3,cost))